Cell 1

In [21]:
# SETUP

# ubah alamat file Tesseract-OCR
folder_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# ubah alamat file browser
folder_browser = "C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"

# ubah alamat file webdriver
folder_webdriver = r"C:\edgedriver_win64\msedgedriver.exe"

# ubah kode mata kuliah dan kelas
# format -> id = [ ['mata kuliah',kelas],  ['mata kuliah 2',kelas], ['mata kuliah 3',kelas], ['mata kuliah 4',kelas] ] 
# contoh -> id = [ ['TKSS213104',2],  ['TKSS213106',2], ['TKSS213107',2], ['TKSS213108',2] ]

# keterangan:
#     mata kuliah = kode mata kuliah, contoh: TKSS213104
#     kelas       = A:1; B:2; C:3
                
id = [ ['TKSS213201',2],  ['TKSS213106',2]]

Cell 2

In [20]:
#  @author Defit Tri H.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.chrome.options import Options
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os, glob, cv2, time
import numpy as np
import pytesseract
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = folder_tesseract

def get_string(img_path):
    # Read image with opencv
    img = cv2.imread(img_path)

    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)

    # Write image after removed noise
    cv2.imwrite("removed_noise.png", img)

    #  Apply threshold to get image with only black and white
    #img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    # Write the image after apply opencv to do some ...
    cv2.imwrite(img_path, img)

    # Recognize text with tesseract for python
    result = pytesseract.image_to_string(Image.open(img_path))

    # Remove template file
    #os.remove(temp)

    return result

def captcha():
    # web screenshot
    output = 'output.png'
    browser.save_screenshot(output)

    # img to numpy array
    capt = cv2.imread(output)
    capt = np.array(capt)

    # crop
    capt_crop = capt[ 130:160, 780:865, :]
    cv2.imwrite('output.png', capt)
    # plt.imshow(capt_crop)

    # image to string OCR
    capt_str = get_string('output.png')

    # filter int from string
    capt_num = ''
    numbers = [str(word) for word in capt_str if word.isdigit()]
    capt_num = capt_num.join(numbers)
    
    try:
        # fill captcha
    #     capt_form = browser.find_element_by_xpath('//*[@id="grid-input-2"]')
        capt_form = browser.find_element("xpath", '//*[@id="grid-input-2"]')
        capt_form.send_keys(capt_num)

        # click submit buttom
    #     submit_but = browser.find_element_by_xpath('//*[@id="f_captcha"]/div[2]/div[1]/button[1]')
        submit_but = browser.find_element("xpath", '//*[@id="f_captcha"]/div[2]/div[1]/button[1]')
        submit_but.click()

        time.sleep(1)
    except:
        print('next')

    # accept alert
    try:
        WebDriverWait(browser, 1).until(EC.alert_is_present())
        alert = browser.switch_to.alert
        alert.accept()
    except:
        print('')


Cell 3: Buka Chrome Webdriver, lalu Login dan Buka laman Pengisian KRS

In [5]:
# edge_options = Options()

# edge_options.add_argument("headless")
# edge_options.binary_location  = folder_browser
browser = webdriver.Edge(executable_path= folder_webdriver)

browser.get("https://simaster.ugm.ac.id")

Cell 4: Auto click + captcha solver

In [ ]:
kode_prodi = browser.find_element("xpath",'/html/body/div[6]/div[2]/div[2]/form/div/div[2]/div/div[4]/table/tbody/tr[2]/td[2]/div')
kode_prodi = kode_prodi.get_attribute("id")[0]
print('kode prodi: ', kode_prodi)

while True:
    for matkul, kelas in id:
        print('kode matkul: ',matkul)
        
        try:
            browser.execute_script("arguments[0].click();", WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="{}_{}"]/table[1]/tbody[1]/tr[{}]/td[1]/input[1]'.format(kode_prodi, matkul, kelas)))))
            time.sleep(1)
            captcha()

            time.sleep(1)
        except:
            time.sleep(1)
            browser.get("https://simaster.ugm.ac.id/sia_krs/input_krs/index1")            

In [ ]:
# close chrome webdriver
browser.close()